In [1]:
import pandas as pd
import numpy as np
from pycaret import classification as cls
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid', font_scale=1)
sns.set_palette('Set2', n_colors=10)
plt.rc('font', family='AppleGothic')
plt.rc('axes', unicode_minus=False)

np.random.seed(1234)
df = pd.read_pickle('data/acc_raw.pkl')
df = df[~df['음주운전'].isnull()]
cat_list = df.select_dtypes(['object']).columns.tolist()
num_list = df.select_dtypes(['number']).drop(columns='음주운전').columns.tolist()

In [2]:
cls.setup(data=df, target='음주운전', train_size=0.7, session_id = 1234)

,Description,Value
0,Session id,1234
1,Target,음주운전
2,Target type,Binary
3,Original data shape,"(59589, 25)"
4,Transformed data shape,"(59589, 114)"
5,Transformed train set shape,"(41712, 114)"
6,Transformed test set shape,"(17877, 114)"
7,Ordinal features,1
8,Numeric features,10
9,Categorical features,14


In [3]:
cls.add_metric('apc', 'APC', average_precision_score, target = 'pred_proba')

Name                                                               APC
Display Name                                                       APC
Score Function       <function average_precision_score at 0x12d98dab0>
Scorer               make_scorer(average_precision_score, needs_pro...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                 True
Multiclass                                                        True
Custom                                                            True
Name: apc, dtype: object

In [4]:
my_models = cls.compare_models(n_select=3, sort='apc')

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:22:44
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


In [5]:
my_models = [cls.tune_model(m) for m in my_models]

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:23:57
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,APC
Fold,,,,,,,,
0,0.9422,0.7764,0.1855,0.5412,0.2763,0.2536,0.2938,0.3504
1,0.9442,0.8093,0.2258,0.5773,0.3246,0.3013,0.3379,0.3448
2,0.9461,0.8326,0.1660,0.6833,0.2671,0.2497,0.3194,0.3770
3,0.9441,0.8052,0.2308,0.5700,0.3285,0.3048,0.3392,0.3472
4,0.9432,0.8569,0.1619,0.5714,0.2524,0.2323,0.2835,0.3707
5,0.9405,0.6734,0.0648,0.4848,0.1143,0.1017,0.1610,0.2185
6,0.9439,0.8337,0.1336,0.6226,0.2200,0.2033,0.2708,0.3477
7,0.9437,0.7920,0.1579,0.5909,0.2492,0.2300,0.2856,0.3208
8,0.9437,0.8375,0.1371,0.6182,0.2244,0.2073,0.2731,0.3615


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:25:13
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Gradient Boosting Classifier


...

Fitting 10 folds for each of 10 candidates, totalling 100 fits


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:26:31
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Extra Trees Classifier


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,APC
Fold,,,,,,,,
0,0.9458,0.8951,0.1371,0.7391,0.2313,0.2167,0.3035,0.4808
1,0.9453,0.8938,0.1250,0.7381,0.2138,0.2000,0.2894,0.4455
2,0.9453,0.8834,0.1215,0.7317,0.2083,0.1948,0.2839,0.4214
3,0.9456,0.8651,0.1215,0.7500,0.2091,0.1958,0.2880,0.4111
4,0.9432,0.8885,0.1053,0.6190,0.1799,0.1656,0.2392,0.4272
5,0.9429,0.8468,0.0891,0.6286,0.1560,0.1434,0.2219,0.3702
6,0.9434,0.8911,0.1093,0.6279,0.1862,0.1717,0.2459,0.4412
7,0.9420,0.8755,0.0850,0.5676,0.1479,0.1345,0.2038,0.4061
8,0.9446,0.8709,0.1008,0.7576,0.1779,0.1663,0.2636,0.4526


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [6]:
cls.plot_model(my_models[0], 'parameter')

,Parameters
bootstrap,True
ccp_alpha,0.0
class_weight,None
criterion,gini
max_depth,None
max_features,sqrt
max_leaf_nodes,None
max_samples,None
min_impurity_decrease,0.0
min_samples_leaf,1


In [7]:
my_stacker = cls.stack_models(my_models, optimize='apc')

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:27:25
Status,. . . . . . . . . . . . . . . . . .,Compiling Estimators
Estimator,. . . . . . . . . . . . . . . . . .,Stacking Classifier
